In [ ]:
from __future__ import division
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from random import shuffle
import time


direct = 'all_bandsCuts100/all_cuts/'
files = os.listdir(direct)
#files = files[0:1000]
all_labels = []
all_images = []
print('loading files...')
for file in files:
    temp = np.load(direct+file)
    all_images += [temp['arr_0']]
    all_labels += [temp['arr_1']]
    

#rescaling everything to 0..1
all_images=np.array(all_images, dtype=np.float32)
all_labels=np.array(all_labels, dtype=np.float32)

print "image shape: ", all_images.shape
imageShape = all_images.shape[1:]

N_channels = len(all_images[0,0,0,:])
maximu = np.zeros(N_channels)
minimu = np.zeros(N_channels)
for i in range(N_channels):
    maximu[i] = np.max(all_images[:,:,:,i])
    minimu[i] = np.min(all_images[:,:,:,i])
    all_images[:,:,:,i] = (all_images[:,:,:,i]-minimu[i])/(maximu[i]-minimu[i])

print('...files loaded')
"""
def make_clts_data(size=1, imageShape = (83,83,8)):
    indices = np.random.randint(all_images.shape[0],size=size)
    ims = np.zeros([size,83,83,8])
    labs = np.zeros([size,83,83,10]) 
    for j in range(len(indices)):
        ims[j,:,:,:] = all_images[indices[j],:,:,:]
        labs[j,:,:,:] = all_labels[indices[j],:,:,:]
    return ims, labs
"""
def make_clts_data(size=1):
    indices = np.random.randint(all_images.shape[0],size=size)
    rotate = np.random.randint(4)
    flip = np.random.randint(1)
    ims = np.zeros([size,83,83,16])
    labs = np.zeros([size,83,83,10])
    for j in range(len(indices)):
        ims[j,:,:,:] = all_images[indices[j],:,:,:]
        labs[j,:,:,:] = all_labels[indices[j],:,:,:]
    ims = np.rot90(ims,k=rotate,axes=(1,2))
    labs = np.rot90(labs,k=rotate,axes=(1,2))
    if flip == 1:
        ims=np.fliplr(ims)
        labs=np.fliplr(labs)
    return ims, labs

def make_clts_test(size=100):
    ims = np.zeros([size,83,83,16])
    labs = np.zeros([size,83,83,10])
    ims = all_images[0:size,:,:,:]
    labs= all_labels[0:size,:,:,:]
    return ims, labs

def weight_variable(shape,name):
    initial = tf.truncated_normal(shape, stddev=0.6)
    return tf.Variable(initial,name=name)

def bias_variable(shape,name):
    initial = tf.constant(-0.1, shape=shape)
    return tf.Variable(initial,name=name)

# factor that controls how much alpha nd beta are updated
memory_factor = 0.999  

def Jacc(activations, truth):
    TPc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum(activations*truth,axis=0),axis=0),axis=0)
    FPc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum(activations*(1-truth),axis = 0),axis=0),axis=0)
    FNc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum((1-activations)*truth,axis = 0),axis=0),axis=0)
    return tf.reduce_mean((TPc + 0.01) / (TPc + FPc + FNc + 2.))

def individualJacc(activations, truth):
    TPc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum(activations*truth,axis=0),axis=0),axis=0)
    FPc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum(activations*(1-truth),axis = 0),axis=0),axis=0)
    FNc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum((1-activations)*truth,axis = 0),axis=0),axis=0)
    return (TPc  + 0.01) / (TPc + FPc + FNc  + 2.)

def alphbet_np(activations, truth):
    global alpha
    global beta

    TPc = np.sum(np.sum(np.sum(activations*truth,axis=0),axis=0),axis=0)
    FNc = np.sum(np.sum(np.sum((1-activations)*truth,axis=0),axis=0),axis=0)
    FPc = np.sum(np.sum(np.sum(activations*(1.-truth),axis=0),axis=0),axis=0)
    alpha = memory_factor*alpha + TPc*(1.-memory_factor)
    beta = memory_factor*beta + (TPc+FNc+FPc)*(1.-memory_factor)
    
def conv2d(xt,Wt):
    return tf.nn.conv2d(xt,Wt,strides=[1,1,1,1], padding='SAME')

In [ ]:
with tf.device('/gpu:0'):

    x = tf.placeholder(tf.float32, [None, None,None,16])
    x_image = x

    N_hidden = 60
    N_hidden3 = 100
    N_feature = 10
    filtSize = 17
    filtSize2 = 1
    """
    W_conv1 = weight_variable([filtSize,filtSize,16,N_hidden],name='W_conv1')
    b_conv1 = bias_variable([N_hidden],name='b_conv1')
    conv_1 = tf.nn.sigmoid(conv2d(x_image, W_conv1)+b_conv1)
    #conv_1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)

    W_conv2 = weight_variable([1,1,N_hidden,N_feature], name='W_conv2')
    b_conv2 = bias_variable([N_feature],name='b_conv2')
    conv_2 = tf.nn.sigmoid(conv2d(conv_1,W_conv2)+b_conv2)
    y_conv = conv_2
    """
    W_conv1 = weight_variable([filtSize,filtSize,16,N_hidden3],name='W_conv1')
    b_conv1 = bias_variable([N_hidden3],name='b_conv1')
    conv_1 = tf.nn.sigmoid(conv2d(x_image, W_conv1)+b_conv1)
    #conv_1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)

    W_conv3 = weight_variable([filtSize2,filtSize2,N_hidden3,N_hidden],name='W_conv3')
    b_conv3 = bias_variable([N_hidden],name='b_conv3')
    conv_3 = tf.nn.sigmoid(conv2d(conv_1, W_conv3)+b_conv3)

    W_conv2 = weight_variable([1,1,N_hidden,N_feature], name='W_conv2')
    b_conv2 = bias_variable([N_feature],name='b_conv2')
    conv_2 = tf.nn.sigmoid(conv2d(conv_3,W_conv2)+b_conv2)
    y_conv = conv_2

    # Define loss and optimizer
    y_truth = tf.placeholder(tf.float32, [None, None, None, N_feature])

alph = tf.placeholder(tf.float32, [10])
bet = tf.placeholder(tf.float32, [10])

cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = y_truth, logits = y_conv))
jacc_loss = -Jacc(y_conv, y_truth, alph, bet)
jacc_loss_reg = jacc_loss + 0.5e-5*tf.reduce_mean(W_conv1*W_conv1) + 0.5e-5*tf.reduce_mean(W_conv2*W_conv2)
indiv_jacc = individualJacc(y_conv,y_truth)

learningRate = tf.placeholder(tf.float32, shape=[])
train_step = tf.train.AdamOptimizer(learningRate).minimize(jacc_loss_reg)

accuracy = tf.reduce_mean(tf.reduce_sum(tf.round(y_truth)*tf.round(y_conv),axis=3)) 




sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
#saver.restore(sess, "/tmp/model2.ckpt")
Learn = 0.001
start = time.time()

for i in range(100000):
    images, labels = make_clts_data(10)
    train_step.run(feed_dict={x:images, y_truth:labels, learningRate: Learn})
    
    if i == 70000:
        Learn = 0.0001
    
    if 0 == (i%10):
        print('step', i)
        testimages, testlabels = make_clts_test()
        predict = y_conv.eval(feed_dict={x: testimages, y_truth:testlabels})
        print('JACCloss TestSet:',jacc_loss.eval(feed_dict={x:testimages, y_truth:testlabels}))
        print('individual JACC',indiv_jacc.eval(feed_dict={x: testimages, y_truth: testlabels}))
        
        print "500 steps took", (time.time()-start)/60
        start = time.time()
testimages, testlabels = make_clts_test()        
final_loss = jacc_loss.eval(feed_dict={x: testimages, y_truth: testlabels})
print('final JACCloss:',final_loss)


save_path = saver.save(sess, "NNET_checkpoint/bmodelr{}.ckpt".format("DeepNNet_subm"))
print("Model saved in file: %s" % save_path)

In [ ]:
Learn = 0.0001
for i in range(1000):
    images, labels = make_clts_data(10)
    alpha = np.zeros(10)
    beta = np.zeros(10)
    train_step.run(feed_dict={x:images, y_truth:labels, learningRate: Learn})
    
    
    
    if 0 == (i%500):
        print('step', i)
        testimages, testlabels = make_clts_test()
        predict = y_conv.eval(feed_dict={x: testimages, y_truth:testlabels})
        print('JACCloss TestSet:',jacc_loss.eval(feed_dict={x:testimages, y_truth:testlabels}))
        print('individual JACC',indiv_jacc.eval(feed_dict={x: testimages, y_truth: testlabels}))
        
from shapely.geometry import MultiPolygon, Polygon
from collections import defaultdict
import tifffile as tiff

def show_mask(m):
    tiff.imshow(255 * np.stack([m, m, m]));


In [ ]:
import csv
import tifffile as tiff
import matplotlib.pyplot
import cv2
print "started"



submIM_IDs = []
cur_dir = '/home/rob/Udacity/capstone/data'
with open(cur_dir + '/sample_submission.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar=',')
    for i,row in enumerate(reader):
        if i == 0:
            i = 1
        if (i%10) == 0:
            submIM_IDs.append(row[0])        

trainIM_IDs = []
with open(cur_dir + '/train_wkt_v4.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar=',')
    for i,row in enumerate(reader):
        if i == 0:
            i = 1
        if (i%10) == 0:
            trainIM_IDs.append(row[0])
#submIM_IDs = trainIM_IDs

N_channels = 16
    
def make_subm_data(size, iteration):
    # Maske hier laden aus Bild und normieren
    
    IM_ID = submIM_IDs[iteration]
    
    im = tiff.imread(cur_dir +'/sixteen_band/{}_A.tif'.format(IM_ID)).transpose([1, 2, 0])
    resized_image_A = cv2.resize(im, (832, 834), interpolation=cv2.INTER_CUBIC)

    im = tiff.imread(cur_dir +'/sixteen_band/{}_M.tif'.format(IM_ID)).transpose([1, 2, 0])
    resized_image_M = cv2.resize(im, (832, 834), interpolation=cv2.INTER_CUBIC)
    
    ims_size = resized_image_M.shape[:2]
    
    image = np.empty((1,834,832,16),dtype=np.float32)
    image[0,:,:,:8] = resized_image_M[:,:,:]
    image[0,:,:,8:16] = resized_image_A[:,:,:] 
    for i in range(N_channels):
        image[0,:,:,i] = (image[0,:,:,i]-minimu[i])/(maximu[i]-minimu[i])
        
    return image



for i in range(len(submIM_IDs)):
    if i%10 == 0:
        print "picture number: ", str(i)
    IM_ID = submIM_IDs[i]
    images = make_subm_data(size=1, iteration=i)
    imageShape = images.shape[1:]
    
    prediction_mask = y_conv.eval(feed_dict={x: images})
    
    np.savez('prediction_masks_16band_031NET/pred{}'.format(IM_ID), prediction_mask)
    

print "job done."